In [1]:
import tensorflow as tf
import numpy as np
import cv2

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
model = tf.keras.models.load_model("model_vgg16")

In [4]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _, frame = cap.read()
    # here is setting the frame to 450 W, 450 H pixels
    frame = frame[50:500, 50:500, :]

    # convert from bgr to rgb due to opencv reads in bgr
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # resize the rgb to 120, 120 which compatible with the model for prediction
    resized = tf.image.resize(rgb, (120, 120))
    # as well as adding the batch and rescaling before predicting
    pred = model.predict(np.expand_dims(resized/255., axis=0))
    # here is to get the predicted coords which is predicting for one image
    sample_coords = pred[1][0]

    # here is to get whether a face is being detected or not over a threshold which is also predicting for one image
    if pred[0] > 0.5:
        # controls the main bounding box
        cv2.rectangle(frame,
                      tuple(np.multiply(sample_coords[:2], [450, 450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450, 450]).astype(int)),
                      (255,0,0),
                      2)

        # controls the label rectangle
        cv2.rectangle(frame,
                      tuple(np.add(np.multiply(sample_coords[:2], [450, 450]).astype(int),
                                   [0, -30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450, 450]).astype(int),
                                   [80, 0])),
                      (255,0,0),
                      -1)

        # this is to put the label on the bounding box
        cv2.putText(frame, "face", tuple(np.add(np.multiply(sample_coords[:2], [450, 450]).astype(int),
                                   [0, -5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow("Face Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()